<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:200%;
           font-family:Arial;letter-spacing:0.5px">

<p width = 20%, style="padding: 10px;
              color:white;">
SQL: Groupby and Joins
              
</p>
</div>

Data Science Cohort Live NYC Nov 2023
<p>Phase 2</p>
<br>
<br>

<div align = "right">
<img src="Images/flatiron-school-logo.png" align = "right" width="200"/>
</div>
    
   

In [1]:
import pandas as pd
import sqlite3
import pandas as pd
conn = sqlite3.connect("data/flights.db")
cur = conn.cursor() 

We've seen aggregations last lecture:

In [ ]:
# Max value for longitude
pd.read_sql('''
    SELECT 
    
         name, MAX(
            CAST(longitude AS REAL) 
        ) AS max_long
    FROM 
        airports
''', conn)

In [ ]:
%%bash

sqlite3 data/flights.db
.schema airlines

In [ ]:
# Effectively counts all the active airlines 
pd.read_sql('''
    SELECT 
        COUNT(*) AS number_of_active_airlines
    FROM 
        airlines
    WHERE 
        active='Y'
''', conn)

This is OK, but want to:
- often compute aggregations across different groups 

#### Grouping in SQL

- Group by values of a categorical column
- Apply aggregation/transformation to groups

GROUP BY statement -- typically used with aggregation!



SELECT colwithgroups,<br>
       agg_function(another_col)<br>
FROM table<br>
GROUP BY colwithgroups<br>

In [ ]:
df_results = pd.read_sql('''
    SELECT *
    FROM 
        airlines
        LIMIT 5
''', conn)
df_results

In [ ]:
df_results = pd.read_sql('''
    SELECT 
        active, COUNT(*) AS number_of_airlines
    FROM 
        airlines
    GROUP BY
        active
''', conn)

df_results

Let's see another example finding the airport with the highest altitude in each country:

In [ ]:
df_results = pd.read_sql('''
    SELECT 
        country,
        name as airport_name, MAX(CAST(altitude AS int)) AS altitude
    FROM 
        airports
    GROUP BY
        country
''', conn)

df_results

Let's look at the head of the airlines table:

In [ ]:
df_results = pd.read_sql('''
    SELECT *
    FROM 
        airlines 
    LIMIT 3
''', conn)

df_results

**Exercise**

Count the number of active flights grouped by country.

In [ ]:
# let's do it!
df_results = pd.read_sql('''

    
''', conn)

df_results

**Exercise**

Get the average airport altitude for each country located at latitude > 30 degrees.

In [ ]:
# Yaaaaaahhhh!
df_results = pd.read_sql('''

    
''', conn)

df_results

**Showcase**: let's get fancy pantsy 

- Combining CASE-WHEN and groupbys

- Partition the airports into three latitude regions: 

- northern (>= 30), central (30,-30), and southern zones (<=-30).

Get me the name, altitude, latitude, and country of the airport in each zone with the maximum altitude.

In [ ]:
# let's do it!
df_results = pd.read_sql('''

SELECT name AS "Airport Name", country,
       MAX(CAST(altitude AS int)) as max_alt,
       CAST(latitude AS float) AS "Latitude (deg)",
       CASE 
            WHEN CAST(latitude AS float) >= 30 THEN "Northern"
            WHEN CAST(latitude AS float) <= -30 THEN "Southern"
            ELSE "Central"
       END lat_category
       
FROM airports

GROUP BY lat_category

    
''', conn)

df_results

#### Filtering Groups with HAVING

- Can filter columns/tables with WHERE (saw this with groupby above)
- WHERE **doesn't** work when filtering **aggregations** on groupby
- `HAVING` will do the trick. 

Let's only return countries with active airlines having a count greater than 30:

In [ ]:
pd.read_sql('''
    SELECT 
        country,
        COUNT(*) AS num
    FROM 
        airlines
    WHERE 
        active='Y'
    GROUP BY 
        country
    HAVING
        num > 30
    ORDER BY 
        num DESC
''', conn)

- Can use multiple aggregation functions in groupby.
- Filter on one of the aggregations.

Get the altitude of the highest airport in a country given that it has at least 100 airports.

In [ ]:
pd.read_sql('''
    SELECT 
        country,
         COUNT(*) AS num_airports,
        MAX(CAST(altitude as int)) as max_altitude
       
    FROM 
        airports
    GROUP BY 
        country
    HAVING
        num_airports >= 100
''', conn)

**Exercise**

List the cities having more than one airport as well as the number of airports the city contains.

Use GROUP BY and HAVING clauses for this.

In [ ]:
pd.read_sql('''


''', conn)

#### Joins
- Saw this in Pandas
- Join information from two tables in a relational database together.
- For the two tables: can relate via a common key (aka foreign key).

SQL has many different kinds of joins. We only look at two:
- INNER JOIN
- LEFT JOIN 

A little picture:

<div>
<center><img src="Images/venn.png" align = "center" width="600"/></center>
</div>

#### Inner joins

- Join two tables together on a shared key. 
- Keep rows if the key is in both tables.


SELECT
    table1.column_name, <br>
    table2.different_column_name <br>
FROM <br>
    table1 <br>
    INNER JOIN table2 <br>
        ON table1.shared_column_name = table2.shared_column_name <br>

##### Syntax: Inner Join
SELECT
    table1.column_name, <br>
    table2.different_column_name <br>
FROM <br>
    table1 <br>
    INNER JOIN table2 <br>
        ON table1.shared_column_name = table2.shared_column_name <br>

Let's check out the airports and airline table. 

In [ ]:
pd.read_sql('''
    SELECT 
        *
    FROM
        routes
    LIMIT 5
    

''', conn)

In [ ]:
pd.read_sql('''
    SELECT 
        *
    FROM
        airlines
        LIMIT 5

''', conn)

In [ ]:
pd.read_sql('''
    SELECT 
        al.name, al.callsign, rt.airline_id, rt.source, rt.dest
    FROM
        airlines AS al
        INNER JOIN
        routes AS rt
        ON al.id = rt.airline_id
        

''', conn)

The airline_id in the routes table and id column in airlines are common keys.

Can join these two tables keeping rows with keys in both tables.

- Note table aliasing

#### LEFT JOIN 

Can join two tables keeping rows with keys in left table only.

In [ ]:
pd.read_sql('''
    SELECT 
        *
    FROM
        airlines as al
        
    LEFT JOIN routes as rt
            ON al.id = rt.airline_id
''', conn)

Can do it the other way:

In [2]:
pd.read_sql('''
    SELECT 
        *
    FROM
        routes as rt
        
    LEFT JOIN airlines as al
            ON rt.airline_id = al.id 
''', conn)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment,index,id,name,alias,iata,icao,callsign,country,active
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2,409.0,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2,409.0,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2,409.0,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2,409.0,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2,409.0,410,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67658,67658,ZL,4178,WYA,6334,ADL,3341,None,0,SF3,4171.0,4178,Regional Express,\N,ZL,RXA,REX,Australia,Y
67659,67659,ZM,19016,DME,4029,FRU,2912,None,0,734,5977.0,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,Y
67660,67660,ZM,19016,FRU,2912,DME,4029,None,0,734,5977.0,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,Y
67661,67661,ZM,19016,FRU,2912,OSS,2913,None,0,734,5977.0,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,Y


**Exercise**

- Get all airports 

In [6]:
pd.read_sql('''
    SELECT 
        *
    FROM
        routes
        
    WHERE source = 'JFK'
''', conn)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,1064,4M,3201,JFK,3797,EZE,3988,Y,0,777
1,1107,4O,17885,JFK,3797,MEX,1824,None,0,320
2,3586,9E,3976,JFK,3797,MSY,3861,None,0,CR9
3,5591,AA,24,JFK,3797,AMM,2170,Y,0,340 330
4,5592,AA,24,JFK,3797,ANU,2874,None,0,738
...,...,...,...,...,...,...,...,...,...,...
451,62537,W3,407,JFK,3797,LOS,273,Y,0,345
452,64593,WS,5416,JFK,3797,YUL,146,None,0,ERJ CRJ
453,64594,WS,5416,JFK,3797,YYC,178,None,0,736
454,64595,WS,5416,JFK,3797,YYZ,193,None,0,CRJ CR7 ERJ


**Exercise**

- Get equipment info for American airline ('AA') routes starting from JFK along with latitude, longitude information for JFK.

In [17]:
pd.read_sql('''
    SELECT 
        *
    FROM
        airports as ap
        
    INNER JOIN routes as rt
            ON ap.code = rt.source
            
    WHERE (source = 'JFK') AND airline = 'AA'
''', conn)

,index,id,name,city,country,code,icao,latitude,longitude,altitude,...,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,3698,3797,John F Kennedy Intl,New York,United States,JFK,KJFK,40.639751,-73.778925,13,...,5591,AA,24,JFK,3797,AMM,2170,Y,0,340 330
1,3698,3797,John F Kennedy Intl,New York,United States,JFK,KJFK,40.639751,-73.778925,13,...,5592,AA,24,JFK,3797,ANU,2874,None,0,738
2,3698,3797,John F Kennedy Intl,New York,United States,JFK,KJFK,40.639751,-73.778925,13,...,5593,AA,24,JFK,3797,AUH,2179,Y,0,777
3,3698,3797,John F Kennedy Intl,New York,United States,JFK,KJFK,40.639751,-73.778925,13,...,5594,AA,24,JFK,3797,AUS,3673,None,0,738
4,3698,3797,John F Kennedy Intl,New York,United States,JFK,KJFK,40.639751,-73.778925,13,...,5595,AA,24,JFK,3797,BCN,1218,None,0,763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,3698,3797,John F Kennedy Intl,New York,United States,JFK,KJFK,40.639751,-73.778925,13,...,5647,AA,24,JFK,3797,TXL,351,Y,0,330
57,3698,3797,John F Kennedy Intl,New York,United States,JFK,KJFK,40.639751,-73.778925,13,...,5648,AA,24,JFK,3797,YUL,146,Y,0,ERD
58,3698,3797,John F Kennedy Intl,New York,United States,JFK,KJFK,40.639751,-73.778925,13,...,5649,AA,24,JFK,3797,YYC,178,None,0,737
59,3698,3797,John F Kennedy Intl,New York,United States,JFK,KJFK,40.639751,-73.778925,13,...,5650,AA,24,JFK,3797,YYZ,193,Y,0,CR7


A lot more unique entries on shared key in the airline table vs. the route table.

We won't go through RIGHT JOIN and FULL JOIN here -- its actually fairly self explanatory what they do.

#### Set Operations

- UNION: gets union, drops duplicates
- UNION ALL: gets union, keeps duplicates
- INTERSECT: gets intersection
- EXCEPT: gets what's in one table but not other.

Useful when you have two tables with same data scheme.

- Table1 has some fields with same kind of data as some fields in Table2.
- Can take intersection of fields from two tables with same kind of data, etc.
- Less flexible than joins, but fast and useful in many cases.

<div>
<center><img src="Images/setopd.png" align = "center" width="600"/></center>
</div>

An example might be useful here:

Get countries in airports table not in airlines table.

Countries that have airports but no airlines.

In [ ]:
pd.read_sql('''
    SELECT ap.country 
    FROM airports as ap
    EXCEPT
    SELECT al.country
    FROM airlines as al
    LIMIT 10
''', conn)

Next up: subqueries!!

Can give us additional flexibility in accessing data!